In [1]:
import MDAnalysis as mda
import nglview as nv
from lipyds.tests.datafiles import NEURONAL_DDAT_GRO, NEURONAL_DDAT_XTC

Let's create our Universe and have a look. This is a coarse-grained universe, so MDAnalysis will warn that it fails to guess masses from the bead names -- we can ignore this for this purpose.

In [2]:
u = mda.Universe(NEURONAL_DDAT_GRO, NEURONAL_DDAT_XTC)
u.add_TopologyAttr("tempfactors")

/Users/lily/pydev/mdanalysis/package/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/lily/pydev/mdanalysis/package/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/lily/pydev/mdanalysis/package/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/lily/pydev/mdanalysis/package/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/lily/pydev/mdanalysis/package/MDAnaly

These data files contain protein, membrane, and solvent (water and ions). The simulations used the coarse-grained MARTINI force field. To have a quick look at the protein and membrane, I select certain common beads to each.

In [3]:
view = nv.show_mdanalysis(u)
view.add_representation('spacefill', '.PO4 or .GL1 or .AM1 or .ROH')
view

## Assigning lipids to a bilayer in a single frame

In [4]:
from lipyds.leafletfinder.leafletfinder import BilayerFinder
from lipyds.leafletfinder.grouping import GraphMethod, SpectralClusteringMethod, GlobalZMethod

### Graph method

This method originates from the MDAnalysis core library itself. It uses `networkx`. Lipids within `cutoff` distance of each other are presumed to be in the same leaflet. Lipids that are not within `cutoff` distance from found leaflets are determined to lie outside them, likely in the midplane.

In [5]:
graph = GraphMethod(
    sparse=False,
    cutoff=10,
    n_leaflets=2
)

finder = BilayerFinder(
    u,
    method=graph,
    select_headgroups="name PO4 GL* AM* ROH"
)
bilayer = finder.run()
bilayer

<Bilayer: lower=561 lipids, upper=622 lipids>

Below we can visualise the found leaflets with NGLView by using the ``bfactors`` or ``tempfactors`` TopologyAttr, and colouring residues by ``bfactor``.

In [6]:
u.residues.atoms.tempfactors = -1
for leaflet in bilayer:
    leaflet.residues.atoms.tempfactors = leaflet._leaflet_index

not_solvent = u.select_atoms("not resname PW ION")

view = nv.show_mdanalysis(not_solvent)
view.add_representation("spacefill", "not protein", color_scheme="bfactor")
view

NGLWidget(max_frame=100)

### Global Z method

We can also try assigning leaflets by distance to the global Z coordinate.

In [7]:
global_z = GlobalZMethod(
    n_leaflets=2,
    cutoff_midplane=4,
)

global_z_finder = BilayerFinder(
    u,
    method=global_z,
    select_headgroups="name PO4 GL* AM* ROH"
)
global_z_bilayer = global_z_finder.run()
global_z_bilayer

<Bilayer: lower=575 lipids, upper=630 lipids>

In [8]:
u.residues.atoms.tempfactors = -1
for leaflet in global_z_bilayer:
    leaflet.residues.atoms.tempfactors = leaflet._leaflet_index

not_solvent = u.select_atoms("not resname PW ION")

cluster_view = nv.show_mdanalysis(not_solvent)
cluster_view.add_representation("spacefill", "not protein", color_scheme="bfactor")
cluster_view

NGLWidget(max_frame=100)

### Spectral clustering method

In [23]:
spectral_clustering = SpectralClusteringMethod(
    delta=10,
    cutoff=20,
    n_leaflets=2,
    angle_factor=1,
    cosine_threshold=1,
)
sc_finder = BilayerFinder(
    u,
    method=spectral_clustering,
    select_headgroups="name PO4 GL* AM* ROH"
)
sc_finder_bilayer = sc_finder.run()
sc_finder_bilayer

<Bilayer: lower=588 lipids, upper=335 lipids>

In [24]:
u.residues.atoms.tempfactors = -1
for leaflet in sc_finder_bilayer:
    leaflet.residues.atoms.tempfactors = leaflet._leaflet_index

not_solvent = u.select_atoms("not resname PW ION")

cluster_view = nv.show_mdanalysis(not_solvent)
cluster_view.add_representation("spacefill", "not protein", color_scheme="bfactor")
cluster_view

NGLWidget(max_frame=100)

## Interacting with leaflets